# 🛡️ EmpathicGateway: The Comprehensive Project Report
### _Engineering Empathy at Scale: AI, Chaos & Resilience_

---

## 📑 Table of Contents
1.  **Executive Summary:** The problem of "Support Fatigue" and our solution.
2.  **System Architecture:** Microservices, Docker, and Hybrid Cloud.
3.  **AI & Model Training:**
    *   Dataset Engineering (Synthetic Injection).
    *   BERT Embeddings + Logistic Regression.
    *   Performance Metrics (Precision/Recall).
4.  **Privacy & Security:** Zero-Trust PII Masking Pipeline.
5.  **Chaos Engineering:** Designing for failure (Backpressure, Semaphores).
6.  **Deployment Saga:** From Google Cloud Run to Synology NAS (Edge).
7.  **Future Roadmap:** LLM Integration and Queue Optimization.



## 1. 🎯 Executive Summary

### The Challenge
Customer support systems are often "First-In, First-Out" (FIFO). This means:
*   A user whose **wallet was stolen** waits in line behind 500 users asking **"How do I change my avatar?"**.
*   Sensitive data (Credit Cards, Phone Numbers) flows unencrypted to third-party chatbots.
*   Traffic spikes cause total system paralysis.

### The Solution: EmpathicGateway
An intelligent API Gateway that:
1.  **Understands Urgency:** Prioritizes Critical requests instantly.
2.  **Protects Privacy:** Redacts PII before it leaves the edge.
3.  **Survives Chaos:** Gracefully degrades service under heavy load instead of crashing.



## 2. 🏗️ System Architecture

Our solution is a **Containerized Microservices** application.

### Tech Stack
| Component | Technology | Role |
| :--- | :--- | :--- |
| **Backend** | Python, FastAPI, PyTorch | The "Brain" (Model Inference, API) |
| **Frontend** | Streamlit, Plotly | The "Control Center" (Traffic Sim, Analytics) |
| **Model** | Sentence-BERT (MiniLM) | Semantic Understanding |
| **Infra** | Docker, Synology NAS | Edge Deployment |
| **Tunnel** | Cloudflare Tunnel | Secure External Access |

### Data Flow Diagram
```mermaid
graph LR
    User[User Request] --> A{Frontend Load Balancer}
    A --> PII[PII Masking Module]
    PII --> Model[Urgency Classifier]
    Model --> Q{Priority Queue}
    Q -->|Critical| Fast[Fast Lane (High Capacity)]
    Q -->|Normal| Slow[Normal Lane (Limited)]
    Slow --> 429[Reject (Overload)]
    Fast --> Agent[LLM / Human Agent]
```



## 3. 🧠 AI & Model Training

This is the core differentiator of EmpathicGateway. We moved beyond simple keyword matching to **Semantic Understanding**.

### A. Dataset Engineering
We started with the **Bitext Customer Support Dataset** (27k samples) but faced a class imbalance.
*   **Problem:** "Lost Wallet" samples were rare.
*   **Solution:** **Synthetic Data Injection**. We manually crafted critical scenarios and "Chit-Chat" samples to teach the model nuance.

```python
# backend/train_model.py
synthetic_data = [
    {"instruction": "my wallet is stolen", "intent": "fraud_report", "priority": 1},
    {"instruction": "unauthorized transaction", "intent": "fraud_report", "priority": 1},
    # ... multiplied by 10x for weight
]

synthetic_normal = [
    {"instruction": "just browsing thanks", "intent": "chit_chat", "priority": 3},
    {"instruction": "hello there", "intent": "greeting", "priority": 3}
]
```

### B. The Model Pipeline (Hybrid Approach)
We prioritized **Inference Speed** (>50ms) over generative capability.
1.  **Embedder:** `sentence-transformers/all-MiniLM-L6-v2` (Converts text to 384-d vectors).
2.  **Classifier:** `LogisticRegression` (Fast, interpretable, effective on high-dimensional vectors).

We custom-built a scikit-learn compatible transformer for BERT:



In [ ]:
# backend/train_model.py
class BertEmbedder(BaseEstimator, TransformerMixin):
    def __init__(self, model_name="all-MiniLM-L6-v2"):
        self.model_name = model_name
        self.model = None

    def fit(self, X, y=None):
        self.model = SentenceTransformer(self.model_name)
        return self

    def transform(self, X):
        return self.model.encode(X.tolist(), show_progress_bar=False)

### C. Performance Metrics
After retraining on the NAS to fix the "Just Browsing" misclassification:

| Metric | Score | Note |
| :--- | :--- | :--- |
| **Accuracy** | **99.84%** | On validation set |
| **F1-Score (Payment Issue)** | 1.00 | Critical detection is solid |
| **F1-Score (Chit Chat)** | 1.00 | Successfully ignores browsing |
| **Inference Time** | <50ms | CPU Only |

The model successfully distinguishes:
*   *"I lost my card"* -> **CRITICAL** (Priority 1)
*   *"I lost the game"* -> **NORMAL** (Priority 3) - *Context Aware!*



## 4. 🛡️ Privacy & Zero-Trust Security

We employ a **Defense-in-Depth** strategy for PII (Personally Identifiable Information).

### Layer 1: Deterministic Regex
Catches high-risk, structured data immediately.
*   **Email:** `[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}`
*   **Credit Card:** `(?:\d[ -]*?){13,16}`

### Layer 2: Contextual NER (Named Entity Recognition)
Using `dslim/bert-base-NER`, we catch unstructured entities:
*   **Names:** "My name is [Murat]" -> `[PERSON]`
*   **Locations:** "I live in [Istanbul]" -> `[LOC]`
*   **Orgs:** "I work at [Google]" -> `[ORG]`

This ensures that even if a user types *"Call me at 555-0199 about my visa"*, the LLM only receives *"Call me at [PHONE] about my [ORG]"*.



## 5. 🔥 Chaos Engineering: Surving the Storm

A key requirement was **Resilience**. We implemented application-level **Backpressure**.

### The Lane System
We define concurrent limits using Asyncio Semaphores (`backend/main.py`).

1.  **FAST LANE (Limit: 10)**
    *   Reserved for: `fraud_report`, `payment_issue`.
    *   Goal: Critical issues *always* get through.

2.  **NORMAL LANE (Limit: 2/5)**
    *   Reserved for: `chit_chat`, `delivery_status`.
    *   Behavior: If >2 requests are active, the 3rd request gets a `429 Too Many Requests`.

### Visual Proof
The Frontend Stress Test module visualizes this in real-time.
*   **Green Bar:** Successful Requests.
*   **Red Bar:** Blocked Requests (Load Shedding).
*   **Result:** The server never crashes; it just sheds excess load strategically.



## 6. 🚀 Deployment Saga

### A. Synology NAS (The Edge)
Deploying modern AI on a NAS brought unique challenges:
1.  **CPU vs GPU:** The NAS lacks NVIDIA GPUs. We optimized the model for CPU inference by removing `cuda`/`mps` dependencies in `fix_model_device.py`.
2.  **Networking:** We bypassed complex router port forwarding using **Cloudflare Tunnel**.
    *   `cloudflared` container runs alongside the app.
    *   Creates a secure outbound tunnel to `trycloudflare.com`.

### B. Docker Composition
We used `docker-compose.synology.yml` to orchestrate the services with `host` networking for optimal performance.

```yaml
services:
  backend:
    image: empathicgateway-backend
    restart: unless-stopped
    volumes:
      - /tmp:/tmp  # For hot-swapping models
  
  tunnel:
    image: cloudflare/cloudflared
    command: tunnel --url http://localhost:8503
    network_mode: host
```



## 7. 📊 Final Stats & Conclusion

### Project Volume
*   **Files:** 40+ Source Files (Python, YAML, Shell)
*   **Lines of Code:** ~2,500+
*   **Documentation:** 7 Detailed Markdown Reports

### Impact
**EmpathicGateway** proves that enterprise-grade resilience and AI intelligence can be delivered in a lightweight, edge-deployable package. We successfully turned a "dumb" pipe into an "empathic" guardian layer for customer support.

---
*Generated for the EmpathicGateway Final Presentation*

